# Code Generator
The requirement: use a Frontier model to generate high performance C++ code from Python code

In [5]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [6]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [7]:
openai = OpenAI()
OPENAI_MODEL = "gpt-4o"
#OPENAI_MODEL = "gpt-4o-mini"


In [8]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [9]:
# for examople here I have write thsi pormpt addign also the what type of class to use because
# without it is not working in  the proper way this is happend for gpt-4o and not for claudi for.  
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [10]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [11]:
# write to a file called optimized.cpp
# Removes code block markers and writes the cleaned C++ code to a file

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [12]:
def optimize_gpt(python):    
    # Makes an API call to OpenAI's chat completion endpoint with the given Python code
    # The 'stream=True' flag allows receiving the response in chunks for efficient handling
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    
    reply = ""
    
    # Iterates through each chunk of the streamed response from OpenAI
    for chunk in stream:
        # Extracts the content of the chunk (delta content) and adds it to the reply
        fragment = chunk.choices[0].delta.content or ""
        
        # Appends the fragment to the reply and prints it in real-time to the console
        reply += fragment
        print(fragment, end='', flush=True)
    
    # Calls write_output function to save the optimized code to a file
    write_output(reply)


In [13]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [14]:
# execution of the file
exec(pi)

Result: 3.141592658589
Execution Time: 12.902812 seconds


In [15]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int iterations, int param1, int param2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        double j1 = static_cast<double>(i * param1 - param2);
        result -= (1.0 / j1);
        double j2 = static_cast<double>(i * param1 + param2);
        result += (1.0 / j2);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();

    double result = calculate(100000000, 4, 1) * 4;

    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> execution_time = end_time - start_time;

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << execution_time.count() << " seconds" << std::endl;

    return 0;
}
```

In [16]:
exec(pi)

Result: 3.141592658589
Execution Time: 13.192239 seconds


In [18]:
python_hard = """
def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [19]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 50.456721 seconds


In [32]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <chrono>

// Linear Congruential Generator
class LCG {
    uint32_t state;
    static constexpr uint32_t a = 1664525;
    static constexpr uint32_t c = 1013904223;
    static constexpr uint32_t m = 1u << 31; // As Python uses 2^32, we use 2^31 to ensure compatibility with signed integers
    
public:
    LCG(uint32_t seed) : state(seed) {}

    uint32_t next() {
        state = (a * state + c) % m;
        return state;
    }
};

int max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    std::vector<int> random_numbers(n);
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = static_cast<int>(lcg.next()) % (max_val - min_val + 1) + min_val;
    }
    
    int max_sum = std::numeric_limits<int>::min();
    for (int i = 0; i < n; ++i) {
        int current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
    

In [33]:
# Replace this with the right C++ compile + execute command for your platform
# to be able to see if it see it will perfomce of c++ compare to python 


#!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
#!./optimized


In [24]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [38]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        convert = gr.Button("Convert code")

    convert.click(stream_gpt, inputs=[python], outputs=[cpp])

ui.launch(inbrowser=True)

In [28]:
# execute the code and gives back the ouptut to see the perfomance 
def execute_python(code):
        try:
            output = io.StringIO()
            sys.stdout = output
            exec(code)
        finally:
            sys.stdout = sys.__stdout__
        return output.getvalue()

In [29]:
# I  need to change the code in the try block to compile the C++ code for your platform
# I pasted this into Claude's chat UI with a request for it to give me a version for an Intel PC,
def execute_cpp(code):
        write_output(code)
        try:
            # run the compile the command 
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
            # 
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            # run the optimizer the coommand 
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)

            return run_result.stdout
            
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [30]:
# css code 
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [40]:
# block is the lower level api of gradio which allows to add css
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])

        
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    # run the optimzie method and the oputu goes to cpp 
    convert.click(stream_gpt, inputs=[python], outputs=[cpp])

    # if the python run is click it will execute the python code run the method execite pytthon and return and in poutptu box the perfomecn e
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    # same but for cpp
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)